In [1]:
import os
if os.name=='nt':
    mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-8.1.0-posix-seh-rt_v6-rev0\\mingw64\\bin'
    os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']

import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import matthews_corrcoef, roc_auc_score
from sklearn.cross_validation import cross_val_score
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold
from sklearn.model_selection import cross_validate, cross_val_predict, StratifiedKFold
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
from sys import getsizeof
import time
import gc

%matplotlib inline

C:\Users\home\Anaconda3\envs\kaggle\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import pickle

def save_pickle(x, filename):
    with open(filename, 'wb') as handle:
        pickle.dump(x, handle, protocol=pickle.HIGHEST_PROTOCOL)

def read_pickle(filename):
    with open(filename, 'rb') as handle:
        x = pickle.load(handle)
    return x

In [3]:
def cross_val_predict_rskf(clf, x_train, y_train, score_callable, n_splits=3, 
                           n_repeats=2, random_state=42):
    '''
    Repeated stratified KFold CV, returns predictions for 
    each repeat and average score.
    n_repeats: repetitions of CV
    '''
    y_pred = []
    scores = []

    n_splits = 3
    n_repeats = 2

    rskf = RepeatedStratifiedKFold(n_repeats=n_repeats, n_splits=n_splits, 
                                   random_state=0)
    np.random.seed(random_state)
    for n, (train_index, test_index) in enumerate(rskf.split(x_train, y_train)):
        x_train_tmp, x_test_tmp = x_train[train_index], x_train[test_index]
        y_train_tmp, y_test_tmp = y_train[train_index], y_train[test_index]

        clf.random_state = np.random.randint(10000000)

        clf.fit(x_train_tmp, y_train_tmp)
        y_pred_tmp = clf.predict_proba(x_test_tmp)[:, 1]
        y_pred.append(y_pred_tmp)
        scores.append(score_callable(y_test_tmp, y_pred_tmp))
        # print('Split {}, score = {:.3f}'.format(n, scores[n]))

    y_pred = np.concatenate([np.concatenate([y_pred[n_splits*i+j] 
                             for j in range(n_splits)]).reshape((-1, 1))
                             for i in range(n_repeats)], axis=1)
    
    score = np.mean(scores)
    
    return y_pred, score

In [21]:
def cross_val_predict_skf_rm(clf, x_train, y_train, score_callable, n_splits=3, 
                           n_repeats=2, random_state=42):
    '''
    Stratified KFold CV with repeated models
    '''
    y_pred_all = []
    scores = []

    n_splits = 3
    n_repeats = 2

    skf = StratifiedKFold(n_splits=n_splits, random_state=0, shuffle=True)
    np.random.seed(random_state)

    for m in range(n_repeats):
        y_pred = []
        print('Repeat {}'.format(m))
        for n, (train_index, test_index) in enumerate(skf.split(x_train, y_train)):
            x_train_tmp, x_test_tmp = x_train[train_index], x_train[test_index]
            y_train_tmp, y_test_tmp = y_train[train_index], y_train[test_index]

            clf.random_state = np.random.randint(10000000)
            # print(clf.random_state)

            clf.fit(x_train_tmp, y_train_tmp)
            y_pred_tmp = clf.predict_proba(x_test_tmp)[:, 1]
            y_pred.append(y_pred_tmp)
            scores.append(score_callable(y_test_tmp, y_pred_tmp))
            print('Split {}, score = {:.3f}'.format(n, scores[m*n_repeats+n]))
        y_pred_all.append(np.concatenate(y_pred).reshape((-1, 1)))
        
    y_pred_all = np.concatenate(y_pred_all, axis=1)
    score = np.mean(scores)
    print('Mean score {:.3f}'.format(score))
    
    return y_pred_all, score

In [5]:
y_train = read_pickle('y_train.pickle')

x_train = read_pickle('x_train_numeric_date_1.pickle')

In [22]:
n_estimators = 100
clf = XGBClassifier(max_depth=14, n_estimators=n_estimators, 
                    base_score=0.0058, n_jobs=-1, colsample_bytree=0.6,
                   min_child_weight=5, subsample=0.9,  
                    reg_lambda=4, silent=False, learning_rate=0.03)

In [ ]:
y_pred, score = cross_val_predict_skf_rm(clf, x_train, y_train, 
                                         roc_auc_score, n_splits=5, 
                                         n_repeats=3, random_state=42)

Repeat 0
